# **(Data Collection)**

## Objectives

* Fetch data from Kaggle and save as raw data.
* Carry out an initial investigation of the data.
* Check data types are correct for carrying out further analysis on the target 'saleprice'.
* Check data types in both csv files are matching so later correlation analysis is accurate.
* Evaluate the data types of the feature variables
* Evaluate the quantity of null values in the dataset and their possible implications.
* Develop an understanding of what distributions and further analysis may be necessary.


## Inputs

* Kaggle JSON file - the authentication token.
* Kaggle dataset URL: codeinstitute/housing-prices-data

## Outputs

* outputs/datasets/collection/original.csv
* outputs/datasets/collection/inherited.csv




---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

* We need to change the working directory from its current folder to its parent folder

    * We access the current directory with os.getcwd()

In [ ]:
import os
current_dir = os.getcwd()
current_dir

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

## Import the dataset from Kaggle.com

- Kaggle must be installed to load the data

- You also need to have registered an account and obtained a Kaggle API key (JSON-File)

In [ ]:
! pip install kaggle==1.5.12

---

- Import operating system
- Set the kaggle config directory to the current working directory
- Set the read and write permissions to read and write for user only (600)

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd()
! chmod 600 kaggle.json

- Define kaggle dataset as the file at the url containing the dataset on kaggle.com
- Set/Create the folder this will stored in to inputs/datasets/raw
- -p flag specifies the save to directory path 
- -d flag specifies the download path 

In [ ]:
KaggleDatasetPath = "codeinstitute/housing-prices-data"
DestinationFolder = "inputs/datasets/raw"
! kaggle datasets download -d {KaggleDatasetPath} -p {DestinationFolder}

In [ ]:
! unzip {DestinationFolder}/*.zip -d {DestinationFolder} \
    && rm {DestinationFolder}/*.zip \
    && rm kaggle.json

## Load and Inspect the Kaggle Data

- The following work contained in this notebook is part of the **'Data Understanding'** phase of the CRISP-DM Workflow.

In [ ]:
import pandas as pd

df_house_prices_records = pd.read_csv(f"inputs/datasets/raw/house-price-20211124T154130Z-001/house-price/house_prices_records.csv")
print(len(df_house_prices_records))
print(df_house_prices_records.shape)
df_house_prices_records.head(8)

#### Check the information relating to each column in the dataframe.
- Note: Any column which has non-null-count below 1460 has 1460-n empty cells
- Note: The target variable is an integer and has no cells with null values
- Note: There are 9 columns which contain missing data
- Note: Enclosed Porch and WoodeckSF have few entries in the dataset
- Note: All columns contain duplicates but there is no concern of having duplicates in their context.

In [ ]:
df_house_prices_records.info()
print(df_house_prices_records['WoodDeckSF'].info())

---

- Check how many unique datatypes are contained in the dataset.

In [ ]:
unique_dtypes = set(df_house_prices_records.dtypes)
unique_dtypes

- Check how many columns contain: **categorical, numerical(int), and numerical(float) variables**

- Return a list of all these columns

In [ ]:
df_house_prices_records_categorical_columns = df_house_prices_records.select_dtypes(include=['object']).columns.to_list()
print(f"There are {len(df_house_prices_records_categorical_columns)} columns that contain categorical data.\n They are: {df_house_prices_records_categorical_columns}.\n\n")

df_house_prices_records_numerical_columns = df_house_prices_records.select_dtypes(include=['int']).columns.to_list()
print(f"There are {len(df_house_prices_records_numerical_columns)} columns that contain numerical data which are integers.\n They are: {df_house_prices_records_numerical_columns}.\n\n")

df_house_prices_records_numerical_fl_columns = df_house_prices_records.select_dtypes(include=['float']).columns.to_list()
print(f"There are {len(df_house_prices_records_numerical_fl_columns)} columns that contain numerical data which are floats.\n They are: {df_house_prices_records_numerical_fl_columns}.\n\n")


In [ ]:

columns_with_nan = df_house_prices_records.columns[df_house_prices_records.isna().sum() > 0].to_list()
columns_with_nan

- Make quick assessment of each of the above features in order to understand what values may be required fill the Nan and 0.0 values if needed.

In [ ]:
# The sum total of 'null' values in all the columns which contain them.
# Interesting observation is the possibility of woodecks and enclosed porches being desirable due to rarity.
df_house_prices_records[columns_with_nan].isna().sum()

In [ ]:
# There are 781 houses from 1460 samples which have no second floor(54% of the data taken)
# Total percentage of houses with no second floor is:
no_top_floor_float = len(df_house_prices_records.query("`2ndFlrSF` == 0.0"))
no_top_floor_NaN = df_house_prices_records['2ndFlrSF'].isna().sum()
print(no_top_floor_float)
print(no_top_floor_NaN)
percen_dataset_no_top_floor = (no_top_floor_float + no_top_floor_NaN)/1460
percen_dataset_no_top_floor = round(percen_dataset_no_top_floor, 2)
print(f"{percen_dataset_no_top_floor}% of houses in this dataset have no top floor")

In [ ]:
# 'BedroomAbvGr' may be a candidate to be transformed to a categorical variable if necessary.
# These are bedrooms that are not at basement level i.e. above ground. Nan could likely be grouped with 0.0 here as both likely mean the same thing in this instance.
unique_bedroom_grades = set(df_house_prices_records['BedroomAbvGr'].unique())
unique_bedroom_grades

In [ ]:
df_unique_bedroom_grades = df_house_prices_records['BedroomAbvGr']
df_unique_bedroom_grades.head(30)

In [ ]:
# Note here: None and nan likely both mean no basement in this context and are essentially the same category.
unique_bsmt_finish_types = set(df_house_prices_records['BsmtFinType1'].unique())
unique_bsmt_finish_types 

In [ ]:
# Enclosed porches will probably add value to a home. They would give more living space and so are possibly a desirable feature.
# The absence of an enclosed porch may also decrease the sale price, and so might correlate to poor sale prices??
# There are 136 examples with porches and 18 unique sizes in that group.
unique_enclosed_porch_values  = set(df_house_prices_records['EnclosedPorch'].unique())
unique_enclosed_porch_values

In [ ]:
# Develop a better understanding of the distribution in this column
unique_garage_finishes = df_house_prices_records['GarageFinish'].unique()
unique_garage_finishes

In [ ]:
# Develop a better understanding of the distribution in this column

df_garage_yt_blt = df_house_prices_records['GarageYrBlt']
df_garage_yt_blt

In [ ]:
df_bsmt_exposure = set(df_house_prices_records['BsmtExposure'].unique())
df_bsmt_exposure

In [ ]:
# Distribution/correlationanalysis needed to understand this feature further.
# NaN values will likely take mean or median
df_lot_frontage = df_house_prices_records['LotFrontage'].head(10)
print(df_lot_frontage)
df = df_house_prices_records['LotFrontage'].iloc[7:8]
df

In [ ]:
# There are only 8 NaN values in this column as seen above. Assess the distribution
df_mas_vnr_area = df_house_prices_records['MasVnrArea']
print(df_mas_vnr_area.head(10))



In [ ]:
# This feature stands out as a possible rarity and something that could add value. May have to alter the NaN to 0.0 to get a measurable square ft value?
df_woodeck = df_house_prices_records['WoodDeckSF']
df_woodeck.head(10)

df = df_house_prices_records['WoodDeckSF'].iloc[7:10]
df.values.sum()

In [ ]:
# Possible room to categorize these variables if required. < 5 = poor > 5 = Good?
df_condition = set(df_house_prices_records['OverallCond'].unique())
print(df_condition)

df_quality = set(df_house_prices_records['OverallQual'].unique())
print(df_quality)


In [ ]:
# Assess later if older houses have any correlation on sale price, particularly if they have been remodelled or have some stand out features.
df_YearBuilt = 	df_house_prices_records['YearBuilt']
print(df_YearBuilt.head(5))

df_YearRemodAdd = df_house_prices_records['YearRemodAdd']
print(df_YearRemodAdd.head(5))


- Study the inherited houses dataset

In [ ]:
df_inherited_houses = pd.read_csv(f"inputs/datasets/raw/house-price-20211124T154130Z-001/house-price/inherited_houses.csv")
df_inherited_houses

- Compare the 2 dataframes to make sure all column data types match up.

In [ ]:
df_inherited_houses.info()

In [ ]:
# See some values which were left out due to length of outputs
df_testing = df_house_prices_records.filter(['WoodDeckSF','YearBuilt','YearRemodAdd'])		
print(df_testing.info())

---

In [ ]:
df_house_prices_records.head(3)

- A comparison of datatypes in the house_records set vs the inherited_houses set highlighted the following needed changes to avoid skewed correlation results further into the project:

    - In the inherited housing set, change the following:
        - BsmtFinSF1 change to int
        - BsmtUnfSF change to int
        - GarageArea change to int
        - TotalBsmtSF change to int
    
    - In the housing records set, change the following:
        - 2ndFlrSF change to int
        - BedroomAbvGr change to int
        - EnclosedPorch change to int
        - WoodDescSF change to int

In [ ]:
columns_to_update1 = ['BsmtFinSF1','BsmtUnfSF', 'GarageArea', 'TotalBsmtSF']

df_inherited_houses[columns_to_update1] = df_inherited_houses[columns_to_update1].astype(int)

df_inherited_houses.info()


In [ ]:
columns_to_update = ['2ndFlrSF', 'BedroomAbvGr', 'EnclosedPorch', 'WoodDeckSF']
df_house_prices_records.loc[:, ('2ndFlrSF', 'BedroomAbvGr', 'EnclosedPorch', 'WoodDeckSF')] = df_house_prices_records[columns_to_update].fillna(0).astype(int)
df_house_prices_records.info()

##### Compare both dataframes for type

- Inherited houses dataframe  dtypes: float64(3), int64(16), object(4)

- House records dataframe  dtypes: float64(3), int64(17), object(4)

In [ ]:
# See which columns remain with NaN values
df_house_prices_records[columns_with_nan].isna().sum()

In [ ]:
# Change the 8 NaN values in MasVnrArea due to small quantity and similar meaning.
df_house_prices_records.loc[:, ('MasVnrArea')] = df_house_prices_records['MasVnrArea'].fillna(0.0)

df_mas_vnr_area = df_house_prices_records['MasVnrArea']
df_mas_vnr_area.isna().sum()

##### These columns still have NaN values at this point.

- BsmtFinType1 (obj: 114)

- GarageFinish (obj: 162)

- GarageYrBlt (float: 81)

- LotFrontage (float: 259)


In [ ]:
remaining_columns_with_nan = ['BsmtFinType1', 'GarageFinish', 'GarageYrBlt', 'LotFrontage']
df_house_prices_records[remaining_columns_with_nan].head(10)

#### Conclusion

- This notebook allowed me to understand:

    -  What datatypes were contained in the dataset
    - The inherited and original set needed to alter some data types so later predictions would be more accurate
    - Some object datatypes which will categorize to numerical if needed.
    - Overall, what data was contained in the dataset and how I might proceed with analysis and cleaning.

# Push files to Repo

* If you do not need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.

In [ ]:
import os
try:
  os.makedirs(name='outputs/datasets/collection')
except Exception as e:
    print(e)

df_house_prices_records.to_csv(f"outputs/datasets/collection/original.csv", index=False)
df_inherited_houses.to_csv(f"outputs/datasets/collection/inherited.csv", index=False)
